# Conversion of s2p to npy

This is used to implement and test the feature suggested by reviewer 2 - allowing easier use of track2p by people using pipelines other than suite2p

Inputs:
- suite2p folder

Outputs:
- rois.npy (bool ndarray of shape: (n_roi, n_px_y, n_px_x))
- fov.npy (float32 ndarray of shape: (n_px_y, n_px_x))            - IMPORTANT: ROIs and FOV must be aligned (this is done by default in suite2p outputs, but might need to be registered manually in other cases)
- traces.npy (float32 ndarray of shape: (n_roi, n_tstamps))


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# these are the paths to the datasets that will be used to 
subject_s2p_path = '/Volumes/data_jm_share/data_proc/jm/jm039/'
# s2p_paths are all the subfolders of subject_path ending in _a
s2p_paths = [f.path for f in os.scandir(subject_s2p_path) if f.is_dir() and f.name.endswith('_a')]
s2p_paths.sort()
print(s2p_paths)

# 
npy_paths = []
for s2p_path in s2p_paths:
    
    # add _npy after jm039 to the path
    npy_path = s2p_path.replace('jm039', 'jm039_npy')
    # make this directory if it does not exist

    if not os.path.exists(npy_path):
        os.makedirs(npy_path)

    npy_paths.append(npy_path)

In [ ]:
# now write code to load the s2p data and saves it to npy
plane = 0

for s2p_path, npy_path in zip(s2p_paths, npy_paths):
    print(f'Loading {s2p_path} and saving to {npy_path}')

    s2p_path_full = os.path.join(s2p_path, 'suite2p', f'plane{plane}')

    ops = np.load(os.path.join(s2p_path_full, 'ops.npy'), allow_pickle=True).item()
    stat = np.load(os.path.join(s2p_path_full, 'stat.npy'), allow_pickle=True)
    iscell = np.load(os.path.join(s2p_path_full, 'iscell.npy'), allow_pickle=True)
    F = np.load(os.path.join(s2p_path_full, 'F.npy'), allow_pickle=True)

    iscell_bool = iscell[:, 0] == 1
    print(sum(iscell_bool), 'cells in this dataset')

    # get mean image
    mean_img = ops['meanImg']
    print('Mean image shape:', mean_img.shape)
    print(type(mean_img[0,0]))
    plt.imshow(mean_img, cmap='gray')
    plt.show()

    # now filter stat and F by iscell_bool
    stat = stat[iscell_bool]
    F = F[iscell_bool, :]
    print('Stat shape:', stat.shape)
    print('F shape:', F.shape)
    print(type(F[0, 0]))

    rois = np.zeros((len(stat), mean_img.shape[0], mean_img.shape[1]), dtype=np.bool)
    for i, s in enumerate(stat):
        # create a mask for the roi
        mask = np.zeros((mean_img.shape[0], mean_img.shape[1]), dtype=np.bool)
        mask[s['ypix'], s['xpix']] = True
        rois[i] = mask
    print('ROIs shape:', rois.shape)
    print(type(rois[0, 0, 0]))

    plt.imshow(rois[0], cmap='gray')
    plt.show()
    plt.imshow(rois[1], cmap='gray')
    plt.show()

    # save to npy
    np.save(os.path.join(npy_path, 'rois.npy'), rois)
    np.save(os.path.join(npy_path, 'F.npy'), F)
    np.save(os.path.join(npy_path, 'fov.npy'), mean_img)

    # TODO: save these to npy files ...
